# 2 数据的使用
## 2.1 去除无用数据
    上周已近已经对数据有了一些了解，直观地了解了数据的基本情况。本周要更加深入地使用数据，首先，要把发现的不连续问题解决。
    经过考虑，选择的解决方案是将不连续的天数数据直接删除，基于这样的假设：短时间内，同一时间的车流量变化不大。

In [1]:
import h5py
import numpy as np
data_Path='../data/TaxiBJ/BJ16_M32x32_T30_InOut.h5'

In [2]:
def del_incomp_date(data_Path):
    reader=h5py.File(data_Path,'r')
    data=reader['data'].value
    date=reader['date'].value
    reader.close()
    data=np.array(data)
    date=np.array(date)
    date_complete=[]##完整的天数
    data_complete=[]
    date_temp=[]##用于统计一天是否有48个时间戳
    data_temp=[]
    date_str=date[0][0:8]
    for i in range(len(date)):
        if date[i][0:8]==date_str:
            date_temp.append(date[i])
            data_temp.append(data[i])
        else :
            if len(date_temp)==48:
                date_complete.extend(date_temp)
                data_complete.extend(data_temp)
            date_temp.clear()
            data_temp.clear()
            date_str=date[i][0:8]
            date_temp.append(date[i])
            data_temp.append(data[i])
    print(len(date_complete),len(data_complete))
    return data_complete,date_complete
data,date = del_incomp_date(data_Path)          

6624 6624


c:\users\hu\miniconda3\envs\pytorch11\lib\site-packages\h5py\_hl\dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


## 2.2 模型的输入
    根据论文，这个模型的依赖4个方面
   * closeness--近期的流量，选择前3个时间戳的数据
   * period--短期的流量，选择前48个时间戳的数据
   * trend--流量预测，提取长时间内流量变化的信息预测，选择前336个时间戳的数据
   * mete--天气情况，只使用当天数据

    因此本周的其他工作是能够输入一个D日期，得到用来计算D的流量的之前日期及其数据

In [3]:
close=[i for i in range(1,4)]
period=[48*j for j in range(1,4)]
trend=[7*48*j for j in range(1,4)]
print(close,period,trend)

[1, 2, 3] [48, 96, 144] [336, 672, 1008]


In [4]:
y=np.asarray(data)

print(y.shape)

(6624, 2, 32, 32)


    可以看出来，要计算一个时间戳的流量数据，最起码要在数据集的1008个时间戳之后才有效

In [5]:
close_data=[]
period_data=[]
trend_data=[]
Y=data[1008:]
time_stamps=[]
close_temp=np.array([])
period_temp=np.array([])
trend_temp=np.array([])
data=np.array(data)
for i in range(1008,(len(data))):
    close_temp=np.vstack([data[i-j] for j in close])
    period_temp=np.vstack([data[i-j] for j in period])
    trend_temp=np.vstack([data[i-j] for j in trend])
    close_data.append(close_temp)
    period_data.append(period_temp)
    trend_data.append(trend_temp)
    time_stamps.append(date[i])
print(np.asarray(close_data).shape)
print(np.asarray(period_data).shape)
print(np.asarray(trend_data).shape)
print(np.asarray(time_stamps).shape)
print(np.asarray(Y).shape)

(5616, 6, 32, 32)
(5616, 6, 32, 32)
(5616, 6, 32, 32)
(5616,)
(5616, 2, 32, 32)


    通过以上步骤，我们获得了有效的、可以用于深度学习的时间戳和当天的流量数据以及用于计算这个流量依赖数据，为了简单起见，我现在先不使用天气的数据


In [7]:
f = h5py.File('../data/TaxiBJ/BJ_Meteorology.h5', 'r')
Timeslot = f['date'].value
WindSpeed = f['WindSpeed'].value
Weather = f['Weather'].value
Temperature = f['Temperature'].value
f.close()

In [8]:
print(Timeslot[0:100])

[b'2013020105' b'2013020106' b'2013020107' b'2013020108' b'2013020109'
 b'2013020110' b'2013020111' b'2013020112' b'2013020113' b'2013020114'
 b'2013020115' b'2013020116' b'2013020117' b'2013020118' b'2013020119'
 b'2013020120' b'2013020121' b'2013020122' b'2013020123' b'2013020124'
 b'2013020125' b'2013020126' b'2013020127' b'2013020128' b'2013020129'
 b'2013020130' b'2013020131' b'2013020132' b'2013020133' b'2013020134'
 b'2013020135' b'2013020136' b'2013020137' b'2013020138' b'2013020139'
 b'2013020140' b'2013020141' b'2013020142' b'2013020143' b'2013020144'
 b'2013020145' b'2013020146' b'2013020147' b'2013020148' b'2013020201'
 b'2013020202' b'2013020203' b'2013020204' b'2013020205' b'2013020206'
 b'2013020207' b'2013020208' b'2013020209' b'2013020210' b'2013020211'
 b'2013020212' b'2013020213' b'2013020214' b'2013020215' b'2013020216'
 b'2013020217' b'2013020218' b'2013020219' b'2013020220' b'2013020221'
 b'2013020222' b'2013020223' b'2013020224' b'2013020225' b'2013020226'
 b'201

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader

In [10]:
class TaxiBJ_Dataset(Dataset):
    def __init__(self,date_Path):
        self.reader,self.date=del_incomp_date(date_Path)
    def __len__(self):
        return len(self.date)
    def __getitem__(self,index):
        day=date[index]
        idx=reader['date'].index(day)
        return reader['data'][index]
        

## 2.4 PyTorch实践
    本周先按照论文所说，尝试模型的建立

In [11]:
from torch import  nn
import torch as t
from torch.nn import  functional as F

In [13]:
class ResUnit(nn.Module):
    def __init__(self,in_flow,out_flow,stride=1):
        super(ResUnit,self).__init__()
        self.left=nn.Sequential(
            nn.Conv2d(in_flow,out_flow,3,stride,1,bias=False),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(out_flow)
            nn.Conv2d(out_flow,out_flow,3,1,1,bias=False),
            nn.BatchNorm2d(out_flow))
        self.right=shortcut
    def forward(self,x):
        out=self.left(x)
        residual=x if self.right is None else self.right(x)
        out+=residual
        return F.relu(out)
class ResNet(nn.Module):
    

SyntaxError: invalid syntax (<ipython-input-13-2df1c21cdaa7>, line 8)